In [1]:
import os
import random
import shutil

# Path to the folder containing the PNG images
folder_path = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia"

# List all files in the folder
file_names = os.listdir(folder_path)

# Shuffle the list of file names randomly
random.shuffle(file_names)

# Calculate the number of files for training (80%) and testing (20%)
num_train = int(0.8 * len(file_names))
num_test = len(file_names) - num_train

# Create directories for training and testing data
train_dir = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\train"
test_dir = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\test"
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Copy 80% of the files to the training directory
for file_name in file_names[:num_train]:
    src_path = os.path.join(folder_path, file_name)
    dst_path = os.path.join(train_dir, file_name)
    shutil.copy(src_path, dst_path)

# Copy 20% of the files to the testing directory
for file_name in file_names[num_train:]:
    src_path = os.path.join(folder_path, file_name)
    dst_path = os.path.join(test_dir, file_name)
    shutil.copy(src_path, dst_path)

print("Data partitioning completed successfully.")

Data partitioning completed successfully.


In [3]:
import cv2
import numpy as np

def add_poisson(img, lambda_val):
    poisson_noise = np.random.poisson(lambda_val, size=img.shape)
    noisy_image = img + poisson_noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

# Function to apply noise to images in a folder
def apply_noise_to_folder(folder_path, output_folder, noise_type, noise_params):
    os.makedirs(output_folder, exist_ok=True)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.png'):
            img_path = os.path.join(folder_path, file_name)
            img = cv2.imread(img_path, 0)
            for param in noise_params:
                noisy_img = None
                if noise_type == 'poisson':
                    noisy_img = add_poisson(img.copy(), param)
                output_subfolder = os.path.join(output_folder, str(param))
                os.makedirs(output_subfolder, exist_ok=True)
                output_path = os.path.join(output_subfolder, file_name)
                cv2.imwrite(output_path, noisy_img)

# Path to the training and test folders
train_folder = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\train"
test_folder = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\test"

# Output folders for noisy training and test sets
noisy_train_folder = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\noisy_train"
noisy_test_folder = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\noisy_test"

# Lambda values for Poisson noise
lambda_values = [25, 50, 75]

# Apply noise to training and test sets
apply_noise_to_folder(train_folder, noisy_train_folder, 'poisson', lambda_values)
apply_noise_to_folder(test_folder, noisy_test_folder, 'poisson', lambda_values)

print("Noise applied to training and test sets.")

Noise applied to training and test sets.


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

# Function to load images from lambda value subfolders within a folder
def load_images_from_folder(folder):
    images = []
    if os.path.isdir(folder):
        subfolders = os.listdir(folder)
        for subfolder in subfolders:
            subfolder_path = os.path.join(folder, subfolder)
            if os.path.isdir(subfolder_path):
                subfolder_images = load_images_from_subfolder(subfolder_path)
                images.extend(subfolder_images)
    return np.array(images)

# Function to load images from a subfolder
def load_images_from_subfolder(subfolder_path):
    images = []
    for filename in os.listdir(subfolder_path):
        img = cv2.imread(os.path.join(subfolder_path, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    return images

# Load clean and noisy images for training and testing
def load_data(train_folder, noisy_train_folder, test_folder, noisy_test_folder):
    clean_train_images = load_images_from_folder(train_folder)
    clean_test_images = load_images_from_folder(test_folder)
    noisy_train_images = load_images_from_folder(noisy_train_folder)
    noisy_test_images = load_images_from_folder(noisy_test_folder)
    return clean_train_images, clean_test_images, noisy_train_images, noisy_test_images

# Normalize the images
def normalize_images(*images):
    normalized_images = []
    for image in images:
        image = image.astype('float32') / 255.
        image = np.expand_dims(image, axis=-1)  # Adding channel dimension
        normalized_images.append(image)
    return normalized_images

# Define the architecture of the CNN autoencoder model
def autoencoder_model(input_shape):
    input_img = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(input_img, decoded)
    return model

# Compile the model
def compile_model(model):
    model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
def train_model(model, noisy_train_images, clean_train_images, noisy_test_images, clean_test_images, epochs=10, batch_size=32):
    model.fit(noisy_train_images, clean_train_images,
              epochs=epochs,
              batch_size=batch_size,
              shuffle=True,
              validation_data=(noisy_test_images, clean_test_images))

# Evaluate the model
def evaluate_model(model, noisy_test_images, clean_test_images):
    score = model.evaluate(noisy_test_images, clean_test_images, verbose=0)
    print("Test loss:", score)

# Define paths to folders
train_folder = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\train"
test_folder = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\test"
noisy_train_folder = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\noisy_train"
noisy_test_folder = r"C:\Users\Dylan Sy\Desktop\biomed\ISCS_30.65\Submission #2\pneumonia\noisy_test"

# Load data
clean_train_images, clean_test_images, noisy_train_images, noisy_test_images = load_data(train_folder, noisy_train_folder, test_folder, noisy_test_folder)

# Normalize images
clean_train_images, clean_test_images, noisy_train_images, noisy_test_images = normalize_images(clean_train_images, clean_test_images, noisy_train_images, noisy_test_images)

# Print shapes for debugging
print("Shapes after normalization:")
print("Clean train images shape:", clean_train_images.shape)
print("Clean test images shape:", clean_test_images.shape)
print("Noisy train images shape:", noisy_train_images.shape)
print("Noisy test images shape:", noisy_test_images.shape)

# Define input shape
input_shape = clean_train_images.shape[1:]

# Define and compile the model
model = autoencoder_model(input_shape)
compile_model(model)

# Train the model
train_model(model, noisy_train_images, clean_train_images, noisy_test_images, clean_test_images)

# Evaluate the model
evaluate_model(model, noisy_test_images, clean_test_images)